In [5]:
import requests
from bs4 import BeautifulSoup
import json

def extractComponent(opinion, selector, attribute=None):
    try:
        if attribute:
            return opinion.select(selector).pop(0)[attribute].strip()
        if attribute is None:
            return opinion.select(selector).pop(0).get_text().strip()
        else:
            return [item.get_text().strip() for item in opinion.select(selector)]            
    except IndexError:
        return None

components = {
    "author": ["span.user-post__author-name"],
    "rcmd": ["span.user-post__author-recomendation > em"],
    "stars": ["span.user-post__score-count"],
    "content": ["div.user-post__text"],
    "pros": ["div[class*=\"positives\"] ~ div.review-feature__item", False],
    "cons": ["div[class*=\"negatives\"] ~ div.review-feature__item", False],
    "purchased": ["div.review-pz"],
    "publishDate": ["span.user-post__published > time:nth-child(1)", "datetime"],
    "purchaseDate": ["span.user-post__published > time:nth-child(2)", "datetime"],
    "useful": ["span[id^='votes-yes']"],
    "useless": ["span[id^='votes-no']"]
}

productId = input("Podaj kod produktu: ")
print('\n')

respons = requests.get("https://www.ceneo.pl/{}#tab=reviews".format(productId))
page = 1
opinionList = []

pageDOM = BeautifulSoup(respons.text, 'html.parser')
opinionCount = int(pageDOM.find_all("span", class_ = "page-tab__title js_prevent-middle-button-click")[2].string.split()[-1].strip('()'))
downloadedOpinions = 0
    
while respons.status_code == 200:
    page += 1

    pageDOM = BeautifulSoup(respons.text, 'html.parser')
    opinions = pageDOM.select("div.js_product-review")

    for opinion in opinions:
        opinionDict = {key:extractComponent(opinion, *value)
                        for key, value in components.items()}
        opinionDict["opinionId"] = opinion["data-entry-id"]
        opinionList.append(opinionDict)
    downloadedOpinions += len(opinions)
    
    print(f"Pobieranie {page} strony z opiniami. Pobrano {round(100 * downloadedOpinions / opinionCount, 0)}%.")
        
    respons = requests.get("https://www.ceneo.pl/{}/opinie-".format(productId)+str(page), allow_redirects=False)

with open(f"./opinions/{productId}.json", "w", encoding="UTF-8") as f:
    json.dump(opinionList, f, indent=4, ensure_ascii=False)

Podaj kod produktu: 22496575


Pobieranie 2 strony z opiniami. Pobrano 5.0%.
Pobieranie 3 strony z opiniami. Pobrano 10.0%.
Pobieranie 4 strony z opiniami. Pobrano 15.0%.
Pobieranie 5 strony z opiniami. Pobrano 21.0%.
Pobieranie 6 strony z opiniami. Pobrano 26.0%.
Pobieranie 7 strony z opiniami. Pobrano 31.0%.
Pobieranie 8 strony z opiniami. Pobrano 36.0%.
Pobieranie 9 strony z opiniami. Pobrano 41.0%.
Pobieranie 10 strony z opiniami. Pobrano 46.0%.
Pobieranie 11 strony z opiniami. Pobrano 52.0%.
Pobieranie 12 strony z opiniami. Pobrano 57.0%.
Pobieranie 13 strony z opiniami. Pobrano 62.0%.
Pobieranie 14 strony z opiniami. Pobrano 67.0%.
Pobieranie 15 strony z opiniami. Pobrano 72.0%.
Pobieranie 16 strony z opiniami. Pobrano 77.0%.
Pobieranie 17 strony z opiniami. Pobrano 82.0%.
Pobieranie 18 strony z opiniami. Pobrano 88.0%.
Pobieranie 19 strony z opiniami. Pobrano 93.0%.
Pobieranie 20 strony z opiniami. Pobrano 98.0%.
Pobieranie 21 strony z opiniami. Pobrano 100.0%.
